In [12]:
import requests
import geopandas as gpd
from shapely.geometry import Polygon
import h3
from lonboard import viz

api_url = "http://localhost:8000/summary"  

aoi = {
    "type": "Polygon",
    "coordinates": [[
        [-74.259090, 40.477399],  
        [-73.700272, 40.477399],  
        [-73.700272, 40.917577],  
        [-74.259090, 40.917577], 
        [-74.259090, 40.477399]   
    ]]
}

payload = {
    "aoi": aoi,
    "spatial_join_method": "centroid",
    "fields": ["sum_pop_f_0_2020",]  
}

response = requests.post(api_url, json=payload)

if response.status_code != 200:
    raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

data = response.json()

# Convert the response data to a GeoDataFrame
geometries = []
for item in data:
    hex_id = item["hex_id"]
    polygon = Polygon(h3.h3_to_geo_boundary(hex_id, geo_json=True))
    geometries.append(polygon)
gdf = gpd.GeoDataFrame(data, geometry=geometries)

viz(gdf)



ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /summary (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x168321550>: Failed to establish a new connection: [Errno 61] Connection refused'))